In [ ]:
#score: 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Conv2D, MaxPooling2D, TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import itertools
import pandas as pd

In [ ]:
# import mnist data and visualize first image
X_train = np.load("/kaggle/input/audio-binary-classification/train_data.npy")
X_test = np.load("/kaggle/input/audio-binary-classification/test_data.npy")
y_train = pd.read_csv('/kaggle/input/audio-binary-classification/train_labels.csv').loc[:,'Label'].values

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
shape1, shape2 = (441, 100)  
X_train = np.reshape(X_train, (-1, shape1, shape2,1))
X_test = np.reshape(X_test, (X_test.shape[0], shape1, shape2,1))
    

In [ ]:
# Tuning the model
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
def build_classifier(filters= 64, add_conv_layers =1, dropout_rate = 0.2, 
                     data_shape=(441, 100),
                     kernel_size=(3,3),
                     recurrent_dropout = 0.2,
                     LSTM_units = 64,
                     add_lstm_layers =1
                    ):
    
    classifier = Sequential()
    shape1, shape2 = data_shape  
    classifier.add(Conv2D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2],1)))
    classifier.add(Dropout(dropout_rate))
    for i in range(add_conv_layers):
        filters //= 2
        classifier.add(Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   activation='relu'))
        classifier.add(MaxPooling2D(pool_size=(2, 2)))
        
    classifier.add(TimeDistributed(Flatten()))
    if(add_lstm_layers):
        classifier.add(LSTM(LSTM_units=LSTM_units, dropout=0.05, recurrent_dropout=recurrent_dropout, return_sequences=True))
        for i in range(add_lstm_layers):
            LSTM_units //= 2
            classifier.add(LSTM(units=LSTM_units, dropout=0.05, recurrent_dropout=recurrent_dropout, return_sequences=False))
        
    else:
        classifier.add(LSTM(units=LSTM_units, dropout=0.05, recurrent_dropout=recurrent_dropout, return_sequences=False))
    
    classifier.add(Dense(units = 1, activation = 'sigmoid'))

    # Compiling
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])

    classifier.summary()
    return classifier

In [ ]:
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [100, 200],
              'epochs': [100, 50],
              'dropout_rate':[0.2, 0.5],
              'filters': [128,64,32],
              'add_conv_layers':[1,2,3],
              'kernel_size':[(3,3),(5,5)],
              'recurrent_dropout':[0.2,0.5],
              'LSTM_units': [64,32],
              'add_lstm_layers': [0,1]
             }
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
# Predicting the Test set results using the best parameters
y_pred = grid_search.predict(X_test)
df2 = pd.DataFrame(y_pred, index = range(y_pred.shape[0]))
df2.reset_index(level=0, inplace=True)
df2.columns=['Id', 'Label']
df2.to_csv("y_pred_conv_LSTM_gridsearch.csv", index=False)
